In [ ]:
import ipywidgets
import pandas as pd
import scispacy
import spacy

# BlueBrainSearch to BlueBrainGraph: POC

This notebook shows how from raw text we can apply BlueBrainSearch and then BlueBrainGraph tools in order to generate first a list of extracted objects of interest, and then a knowledge graph out of it.

It is intended to be just a proof of concept of the pipeline.

## BlueBrainSearch

This first part of the pipeline starts with the raw text of a scientific paper as an input, and generates a CSV table out of it. The table contains all the extracted entities and relations that were identified in the text.
- **input**: raw text
- **output**: csv table of extracted entities/relations

In [ ]:
NER_PIPELINE = spacy.load("en_ner_craft_md")

In [ ]:
from collections import OrderedDict
import IPython

bbs_widgets = OrderedDict()
bbs_widgets['input_text'] = ipywidgets.Textarea(
        value="Autophagy maintains tumour growth through circulating arginine. Autophagy captures intracellular components and delivers them to lysosomes, where they are degraded and recycled to sustain metabolism and to enable survival during starvation1-5. Acute, whole-body deletion of the essential autophagy gene Atg7 in adult mice causes a systemic metabolic defect that manifests as starvation intolerance and gradual loss of white adipose tissue, liver glycogen and muscle mass1. Cancer cells also benefit from autophagy. Deletion of essential autophagy genes impairs the metabolism, proliferation, survival and malignancy of spontaneous tumours in models of autochthonous cancer6,7. Acute, systemic deletion of Atg7 or acute, systemic expression of a dominant-negative ATG4b in mice induces greater regression of KRAS-driven cancers than does tumour-specific autophagy deletion, which suggests that host autophagy promotes tumour growth1,8. Here we show that host-specific deletion of Atg7 impairs the growth of multiple allografted tumours, although not all tumour lines were sensitive to host autophagy status. Loss of autophagy in the host was associated with a reduction in circulating arginine, and the sensitive tumour cell lines were arginine auxotrophs owing to the lack of expression of the enzyme argininosuccinate synthase 1. Serum proteomic analysis identified the arginine-degrading enzyme arginase I (ARG1) in the circulation of Atg7-deficient hosts, and in vivo arginine metabolic tracing demonstrated that serum arginine was degraded to ornithine. ARG1 is predominantly expressed in the liver and can be released from hepatocytes into the circulation. Liver-specific deletion of Atg7 produced circulating ARG1, and reduced both serum arginine and tumour growth. Deletion of Atg5 in the host similarly regulated [corrected] circulating arginine and suppressed tumorigenesis, which demonstrates that this phenotype is specific to autophagy function rather than to deletion of Atg7. Dietary supplementation of Atg7-deficient hosts with arginine partially restored levels of circulating arginine and tumour growth. Thus, defective autophagy in the host leads to the release of ARG1 from the liver and the degradation of circulating arginine, which is essential for tumour growth; this identifies a metabolic vulnerability of cancer. (PMID:30429607)",
        layout=ipywidgets.Layout(width='80%', height='400px')
    )
bbs_widgets['submit_button'] = ipywidgets.Button(
    description='Extract Entities & Properties!',
    layout=ipywidgets.Layout(width='30%')
)

bbs_widgets['out'] = ipywidgets.Output(layout={'border': '0.5px solid black'})

In [ ]:
def extract_entities_and_relations(b):
    bbs_widgets['out'].clear_output()
    with bbs_widgets['out']:
        doc = NER_PIPELINE(bbs_widgets['input_text'].value)
        lines = []
        for ent in doc.ents:
            lines.append(pd.Series({'entity': ent.text,
                                    'entity_type': ent.label_,
                                    'property':None,
                                    'property_value':None,
                                    'property_type':None,
                                    'property_value_type':None,
                                    'ontology_source':'MyOntology',
                                    'paper_id':0,
                                    'start_pos': ent.start_char,
                                    'end_pos': ent.end_char,
                                    }))

        df = pd.DataFrame(lines)
        display(df)

bbs_widgets['submit_button'].on_click(extract_entities_and_relations)

ordered_widgets = list(bbs_widgets.values())
main_widget = ipywidgets.VBox(ordered_widgets)
IPython.display.display(main_widget)

## BlueBrainGraph

This second part of the pipeline starts with the extracted entities and relations in a CSV table, and generates a knowledge graph out of it.

- **input**: csv table of extracted entities/relations
- **output**: knowledge graph